In [10]:
import numpy as np
import cv2 as cv

In [11]:
img = cv.cvtColor(cv.imread("picture2.jpg"),cv.COLOR_BGR2RGB)
img.shape

(4000, 6016, 3)

In [3]:
def bin_image(image,dimension=24):
    output = np.empty(list([dimension])+list(image.shape[:2]))
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            temp = list("".join([format(x, 'b').zfill(8) for x in image[i,j]]))
            output[:,i,j] = [int(x) for x in temp]
    return output

In [3]:
def hv_read1(matrix):
    horizontal_clues = [nono_creator1(matrix[i]) for i in range(matrix.shape[0])]
    vertical_clues = [nono_creator1(matrix[:,i]) for i in range(matrix.shape[1])]
    return horizontal_clues, vertical_clues

In [4]:
def nono_creator1(arr):
    output = []
    count = 0
    for element in arr:
        if element == 1:
            count += 1
        elif count > 0:
            output.append(count)
            count = 0
    if count > 0:
        output.append(count)
    return output

In [ ]:
def grids_values(num):
    grid5 = 0
    grid10 = 0
    grid20 = num//400
    remainder = num%400
    print("20grid:",grid20,"\t","remainder:",remainder)
    if remainder > 300:
        adding_value = 400-remainder
        remainder = 0
        grid20 += 1
        print("20grid:",grid20,"\t","adding value:",adding_value)
    else:
        grid10 = remainder//100
        remainder = remainder%100
        print("10grid:",grid10,"\t","remainder:",remainder)
        if remainder > 60:
            adding_value = 100-remainder
            remainder = 0
            grid10 += 1
            print("10grid:",grid10,"\t","adding value:",adding_value)
        else:
            grid5 = remainder//25
            remainder = remainder%25
            adding_value = 0
            print("5grid:",grid5,"\t","adding value:",adding_value)
    out = np.cumsum([grid20*400,grid10*100,grid5*25])
    return (np.concatenate((out,(adding_value,-remainder))),(grid20,grid10,grid5))

In [ ]:
out = bin_image(img).reshape(-1)

In [ ]:
def grids_values(num):
    grid5 = 0
    grid10 = 0
    adding_value = 0
    grid20 = num // 400
    remainder = num % 400
    print("20grid:", grid20, "\t", "remainder:", remainder)
    if remainder > 300:
        adding_value = 400 - remainder
        remainder = 0
        grid20 += 1
        print("20grid:", grid20, "\t", "adding value:", adding_value)
    else:
        grid10 = remainder // 100
        remainder = remainder % 100
        print("10grid:", grid10, "\t", "remainder:", remainder)
        if remainder > 60:
            adding_value = 100 - remainder
            remainder = 0
            grid10 += 1
            print("10grid:", grid10, "\t", "adding value:", adding_value)
        else:
            grid5 = remainder // 25
            remainder = remainder % 25
            if remainder > 15:
                adding_value = 25 - remainder
                remainder = 0
                grid5 += 1
            print("5grid:", grid5, "\t", "adding value:", adding_value)
        print("5grid:", grid5, "\t", "remainder:", remainder)
        print(remainder)
    return np.cumsum([grid20 * 400, grid10 * 100, grid5 * 25]),adding_value,num-remainder+adding_value

In [ ]:
values, grids = grids_values(out.shape[0])

In [ ]:
mat20 = out[:values[0]].reshape(-1,20,20)

In [ ]:
mat10 = out[values[0]:values[1]].reshape(-1,10,10)

In [ ]:
mat5 = out[values[1]:values[2]].reshape(-1,5,5)

In [42]:
class GridsClues:
    def __init__(self, path, dimentions=None):
        self.image = cv.cvtColor(cv.imread(path), cv.COLOR_BGR2RGB)
        self.image_height, self.image_weight = self.image.shape[:2]
        self.dimension = dimentions if dimentions else 24
        self.remainder_values = None
        self.adding_value = None
        self.mat20 = None
        self.mat10 = None
        self.mat5 = None
        self.mat20_clues = None
        self.mat10_clues = None
        self.mat5_clues = None
        self.flow()

    def _bin_image(self):
        output = np.empty(list([self.dimension]) + [self.image_height, self.image_weight])
        for i in range(self.image_height):
            for j in range(self.image_weight):
                temp = list("".join([format(x, 'b').zfill(8) for x in self.image[i, j]]))
                output[:, i, j] = [int(x) for x in temp]
        return output

    def _hv_read(self, matrix):
        horizontal_clues = [self._nono_creator(matrix[i]) for i in range(matrix.shape[0])]
        vertical_clues = [self._nono_creator(matrix[:, i]) for i in range(matrix.shape[1])]
        return horizontal_clues, vertical_clues

    def _nono_creator(self, arr):
        output = []
        count = 0
        for element in arr:
            if element == 1:
                count += 1
            elif count > 0:
                output.append(count)
                count = 0
        if count > 0:
            output.append(count)
        return output

    def _grids_values(self):
        num = self.image_height * self.image_weight * self.dimension
        grid5 = 0
        grid10 = 0
        adding_value = 0
        grid20 = num // 400
        remainder = num % 400
        if remainder > 300:
            adding_value = 400 - remainder
            remainder = 0
            grid20 += 1
        else:
            grid10 = remainder // 100
            remainder = remainder % 100
            if remainder > 60:
                adding_value = 100 - remainder
                remainder = 0
                grid10 += 1
            else:
                grid5 = remainder // 25
                remainder = remainder % 25
                if remainder > 15:
                    adding_value = 25 - remainder
                    remainder = 0
                    grid5 += 1
        self.num_mat20, self.num_mat10, self.num_mat5, self.adding_value = grid20, grid10, grid5, adding_value
        return np.cumsum([grid20 * 400, grid10 * 100, grid5 * 25]), num - remainder

    def flow(self):
        bin_out = self._bin_image().reshape(-1)
        values, output_range = self._grids_values()
        if self.adding_value:
            bin_out = np.concatenate((bin_out, np.ones(self.adding_value)))
        elif output_range:
            self.remainder_values = bin_out[output_range:]
            bin_out = bin_out[:output_range]

        self.mat20 = bin_out[:values[0]].reshape(-1, 20, 20)
        index = []
        for i in range(self.mat20.shape[0]):
            if np.sum(self.mat20[i])>0:
                index.append(i)
        self.mat20 = self.mat20[index]
#         self.mat10 = bin_out[values[0]:values[1]].reshape(-1, 10, 10)
#         self.mat5 = bin_out[values[1]:values[2]].reshape(-1, 5, 5)

        clues = [self._hv_read(x) for x in self.mat20]
        self.mat20_clues = np.array([clues_to_matrix(x) for x in clues])
#         self.mat10_clues = [self._hv_read(x) for x in self.mat10]
#         self.mat5_clues = [self._hv_read(x) for x in self.mat5]
        print("all Done")

In [43]:
obj1 = GridsClues("picture.jpg")

all Done


In [44]:
matrix,clues = obj1.mat20,obj1.mat20_clues

In [5]:
def clues_to_matrix(clues):
    clues = clues[0]+clues[1]
    output = np.zeros((40,10))
    for i in range(40):
        output[i,:len(clues[i])] = clues[i]
    return output

In [6]:
matrix.shape

(10410, 20, 20)

In [8]:
clues = [clues_to_matrix(x) for x in clues]

In [11]:
clues = np.array(clues)

In [12]:
clues.shape

(10410, 40, 10)

In [26]:
matrix_arr = []
clues_arr = []
total_sum = []
for i in range(clues.shape[0]):
    tsum = np.sum(clues[i])
    if tsum > 0:
        matrix_arr.append(matrix[i])
        clues_arr.append(clues[i])
        total_sum.append(tsum)
    if not i%1000:
        print("working...")

matrix_arr = np.array(matrix_arr)
clues_arr = np.array(clues_arr)

working...
working...
working...
working...
working...
working...
working...
working...
working...
working...
working...


In [27]:
matrix_arr.shape

(9791, 20, 20)

In [28]:
clues_arr.shape

(9791, 40, 10)

In [31]:
total_sum

[2.0,
 16.0,
 120.0,
 194.0,
 258.0,
 328.0,
 194.0,
 236.0,
 470.0,
 530.0,
 330.0,
 304.0,
 494.0,
 592.0,
 346.0,
 348.0,
 470.0,
 582.0,
 328.0,
 396.0,
 446.0,
 582.0,
 318.0,
 418.0,
 404.0,
 558.0,
 294.0,
 456.0,
 382.0,
 562.0,
 292.0,
 418.0,
 378.0,
 512.0,
 236.0,
 306.0,
 386.0,
 440.0,
 210.0,
 234.0,
 396.0,
 398.0,
 204.0,
 194.0,
 402.0,
 402.0,
 234.0,
 172.0,
 410.0,
 410.0,
 264.0,
 156.0,
 408.0,
 418.0,
 294.0,
 162.0,
 386.0,
 422.0,
 322.0,
 166.0,
 362.0,
 428.0,
 350.0,
 172.0,
 338.0,
 434.0,
 378.0,
 178.0,
 316.0,
 438.0,
 408.0,
 184.0,
 292.0,
 444.0,
 436.0,
 190.0,
 268.0,
 450.0,
 452.0,
 208.0,
 246.0,
 456.0,
 458.0,
 238.0,
 222.0,
 462.0,
 462.0,
 266.0,
 206.0,
 458.0,
 468.0,
 294.0,
 214.0,
 434.0,
 474.0,
 324.0,
 218.0,
 412.0,
 478.0,
 352.0,
 222.0,
 388.0,
 484.0,
 380.0,
 230.0,
 364.0,
 488.0,
 408.0,
 234.0,
 340.0,
 492.0,
 436.0,
 234.0,
 314.0,
 492.0,
 462.0,
 234.0,
 288.0,
 492.0,
 488.0,
 234.0,
 262.0,
 492.0,
 492.0,
 256.0,
 23

In [6]:
%%time
img = cv.imread("picture.jpg")
img.shape

CPU times: total: 0 ns
Wall time: 3 ms


(328, 529, 3)

In [18]:
def bin_image(image,dimension=24):
    image = image.reshape(-1,3)
    output = np.empty(list([dimension])+[image.shape[0]])
    for i in range(image.shape[0]):
            temp = list("".join([format(x, 'b').zfill(8) for x in image[i]]))
            output[:,i] = [int(x) for x in temp]
    return output
def hv_read2(matrix):
    clues = np.zeros((40,10))
    for i in range(20):
        clues[i] = nono_creator2(matrix[i])
        clues[i+20] = nono_creator2(matrix[:,i])
    return clues
def clues_to_matrix(clues):
    clues = clues[0]+clues[1]
    output = np.zeros((40,10))
    for i in range(40):
        output[i,:len(clues[i])] = clues[i]
    return output
def _nono_creator(self, arr):
        output = []
        count = 0
        for element in arr:
            if element == 1:
                count += 1
            elif count > 0:
                output.append(count)
                count = 0
        if count > 0:
            output.append(count)
        return np.array(output,dtype=np.uint8)

In [19]:
%%time
imgout = bin_image(img)

CPU times: total: 1.22 s
Wall time: 1.24 s


In [20]:
shape = imgout.shape[1]-imgout.shape[1]%400

In [21]:
imgout =imgout[:,:shape].reshape(-1,20,20)

In [22]:
%%time
for i in imgout:
    hv_read2(i)

ValueError: could not broadcast input array from shape (11,) into shape (10,)

In [23]:
%%time
for x in imgout:
    x = hv_read1(x)
    clues = clues_to_matrix(x)

CPU times: total: 2.41 s
Wall time: 2.42 s


In [5]:
r = 256*256*256
a = {}
for i in range(r):
    a[i] = list(format(i, 'b').zfill(8))

In [8]:
a[0]

['0', '0', '0', '0', '0', '0', '0', '0']

In [8]:
def fun(image,dimension=24):
    output = np.empty(list([dimension])+list(image.shape[:2]))
    r = 256
    a = {}
    for i in range(r):
        a[i] = list(format(i, 'b').zfill(8))
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            temp = []
            for x in image[i,j]:
                temp.extend(a[x])
            output[:,i,j] = [int(x) for x in temp]
    return output

In [5]:
def bin_image(image,dimension=24):
    output = np.empty(list([dimension])+list(image.shape[:2]))
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            temp = list("".join([format(x, 'b').zfill(8) for x in image[i,j]]))
            output[:,i,j] = [int(x) for x in temp]
    return output

In [12]:
%%time
fun(img)

CPU times: total: 4min 2s
Wall time: 4min 2s


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 1., 1.],
        [0., 0., 0., ..., 1., 1., 1.],
        [0., 0., 0., ..., 1., 1., 1.]],

       [[1., 1., 0., ..., 1., 1., 1.],
        [0., 1., 0., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [0., 1., 0., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 0., 1., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 1., 1., 1.],
        [1., 0., 1., ..., 1., 1., 0.],
        [0., 0., 0., ..., 1., 1., 1.],
        ...,
        [1., 0., 1., ..., 1., 1., 1.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 1., 0., ..., 0., 1., 0.]],

       ...,

       [[0., 1., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 0., 1.],
        ...,
        [0., 1., 0., ..., 0., 0., 1.],
        [0., 0., 1., ..., 1., 0., 0.],
        [0., 0., 1., ..., 1., 0.

In [13]:
%%time
bin_image(img)

CPU times: total: 2min 48s
Wall time: 2min 48s


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 1., 1.],
        [0., 0., 0., ..., 1., 1., 1.],
        [0., 0., 0., ..., 1., 1., 1.]],

       [[1., 1., 0., ..., 1., 1., 1.],
        [0., 1., 0., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [0., 1., 0., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 0., 1., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 1., 1., 1.],
        [1., 0., 1., ..., 1., 1., 0.],
        [0., 0., 0., ..., 1., 1., 1.],
        ...,
        [1., 0., 1., ..., 1., 1., 1.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 1., 0., ..., 0., 1., 0.]],

       ...,

       [[0., 1., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 0., 1.],
        ...,
        [0., 1., 0., ..., 0., 0., 1.],
        [0., 0., 1., ..., 1., 0., 0.],
        [0., 0., 1., ..., 1., 0.